In [ ]:
import os
import requests

In [ ]:
%run ../common/data-files.py

In [ ]:
def get_venue_ids():
    venues_df = DataFiles.read('ticketbasher/venues')
    
    active_venues = venues_df[venues_df['active'] == True]
    
    return list(active_venues['id'])

In [ ]:
def get_seatgeek_events_by_venues(venue_ids):
    url = 'https://api.seatgeek.com/2/events'
    params = {
        'client_id': os.environ.get('SEATGEEK_CLIENT_ID'),
        'taxonomies.name': 'concert',
        'sort': 'score.desc',
        'listing_count.gt': 0,
        'venue.id': ','.join(str(x) for x in venue_ids),
        'per_page': 250
    }

    events = requests.get(url=url, params=params)
    
    return events.json()

In [ ]:
def parse_events(events):
    parsed_events = []

    for event in events['events']:
        primary_artist = [artist for artist in event['performers'] if ('primary' in artist) and (artist['primary'] == True)][0]

        no_primary_genre = {
            'id': None,
            'slug': None
        }
        if 'genres' in primary_artist:
            primary_genres = [genre for genre in primary_artist['genres'] if genre['primary'] == True]
            if primary_genres:
                primary_genre = primary_genres[0]
            else:
                primary_genre = no_primary_genre
        else:
            primary_genre = no_primary_genre
                
        parsed_events.append({
            'id': event['id'],
            'title': event['title'],
            'datetime_utc': event['datetime_utc'],
            'popularity': event['popularity'],
            'venue_id': event['venue']['id'],
            'primary_artist_id': primary_artist['id'],
            'primary_artist_slug': primary_artist['slug'],
            'primary_genre_id': primary_genre['id'],
            'primary_genre_slug': primary_genre['slug'],
            
            'listing_count': event['stats']['listing_count'],
            'visible_listing_count': event['stats']['visible_listing_count'],
            'lowest_price': event['stats']['lowest_price'],
            'highest_price': event['stats']['highest_price'],
            'average_price': event['stats']['average_price'],
            'median_price': event['stats']['median_price'],
        })

    return parsed_events

In [ ]:
def main():
    # Get venues in Portland
    venue_ids = get_venue_ids()

    # Get events in Portland
    portland_events = get_seatgeek_events_by_venues(venue_ids)
    
    # Parse events
    parsed_portland_events = parse_events(portland_events)

    # Write dataset
    DataFiles.write('ticketbasher/events', pd.DataFrame(parsed_portland_events))

In [ ]:
main()